In [1]:
#  相關套件

import time
import random
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
from collections import deque
from concurrent.futures import ThreadPoolExecutor
import urllib.parse
import urllib3

# 忽略不安全請求的警告
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

WEB_NAME = "1111_人力銀行"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36",
    "Referer": "https://www.1111.com.tw",
}

print(f"開始執行 {WEB_NAME} ")

開始執行 1111_人力銀行 


In [2]:
# ## 取得網站所有職業總覽
# # 1. 取得 JSON 資料
# # jobcat 檔案名稱

# file_jobcat_json = f"{WEB_NAME}_jobcat_json.txt"
# url_JobCat = "https://www.1111.com.tw/api/v1/codeCategories/"

# response_jobcat = requests.get(url_JobCat, headers=HEADERS, verify=False, timeout=10)
# response_jobcat.raise_for_status()
# jobcat_data = response_jobcat.json()
# with open(file_jobcat_json, "w", encoding="utf-8") as f:
#     json.dump(jobcat_data, f, ensure_ascii=False, indent=4)

# print("JSON資料，分類總覽", pd.json_normalize(jobcat_data).columns)
# # 'nation', 'city', 'region', 'mrt', 'jobPosition', 'industry',
# # 'certification', 'major', 'workAbility', 'computerSkill',
# # 'companyBenefit', 'jobBenefit',
# # pd.json_normalize(jobcat_data)
# # pd.json_normalize(jobcat_data["region"][0]["categories"])
# # pd.json_normalize(jobcat_data["certification"])
# # pd.json_normalize(jobcat_data["major"])
# # pd.json_normalize(jobcat_data["workAbility"])
# # pd.json_normalize(jobcat_data["computerSkill"])
# print(f"職業總覽資料已儲存為 {file_jobcat_json}")

# df_jobcat = pd.json_normalize(jobcat_data["industry"])
# df_jobcat.to_excel(f"{WEB_NAME}_category.xlsx", index=False)
# print(f"職業總覽資料已轉換為 '{WEB_NAME}_category.xlsx'")

# mask = df_jobcat["parentCode"].astype(str).str.startswith("100")
# df_it_jobs = df_jobcat[mask]
# df_it_jobs.head(5)

In [3]:
import urllib.parse


def catch_1111_url(KEYWORDS, CATEGORY, ORDER="date", PAGE_NUM=1, USE_API=False):
    """
    這個函數會根據給定的關鍵字、類別、排序和頁碼參數，
    構建一個 1111 求職網的完整職缺網址或 API 網址。

    參數:
    KEYWORDS (str): 職缺的關鍵字，例如 "雲端工程師"。若無則傳入空字串 ""。
    CATEGORY (str or list): 職缺的類別代碼，例如 "140100" 或者類別代碼的列表。
                            若無則傳入空字串 ""。
    ORDER (str, optional): 排序方式。可選值為 "relevance" (相關性) 或 "date" (最新日期)。
                           預設為 "date"。
    PAGE_NUM (int, optional): 指定的頁碼。預設為 1。
    USE_API (bool, optional): 是否使用 API 網址。預設為 False。

    返回:
    str: 生成的 1111 求職網址或 API 網址。
    """
    BASE_URL = "https://www.1111.com.tw/search/job"
    API_URL = "https://www.1111.com.tw/api/v1/search/jobs/"

    # 確保頁碼至少為 1，避免負數或 0 造成計算錯誤
    safe_page_num = max(1, PAGE_NUM)

    if USE_API:
        params = {
            "page": safe_page_num,
            "fromOffset": 0,
            "sortBy": "ab" if ORDER == "relevance" else "da",
            "sortOrder": "desc",
            "conditionsText": KEYWORDS,
            "searchUrl": f"/search/job?page={safe_page_num}&col={'ab' if ORDER == 'relevance' else 'da'}&sort=desc&ks={urllib.parse.quote(KEYWORDS)}&d0={','.join(CATEGORY) if isinstance(CATEGORY, list) else CATEGORY}",
            "keyword": KEYWORDS,
        }

        # 如果有提供職務類別，加入到參數中
        if CATEGORY:
            if isinstance(CATEGORY, list):
                for code in CATEGORY:
                    params[f"jobPositions"] = code

        query_string = urllib.parse.urlencode(params)
        return f"{API_URL}?{query_string}"
    else:
        params = {
            "page": safe_page_num,
            "col": "ab" if ORDER == "relevance" else "da",
            "sort": "desc",
            "ks": KEYWORDS,
            "d0": ",".join(CATEGORY) if isinstance(CATEGORY, list) else CATEGORY,
        }

        query_string = urllib.parse.urlencode(params)
        return f"{BASE_URL}?{query_string}"


# # --- 測試範例 ---

# KEYWORDS_STR = "雲端工程師"
# CATEGORY_CODE1 = "140100"  # 單一類別
# CATEGORY_CODE2 = ["140100", "140200"]  # 多個類別

# # 1. 有關鍵字, 單一類別, 相關性排序, 第 1 頁
# print("1. 有關鍵字, 單一類別, 相關性排序, 第 1 頁:")
# url_1 = catch_1111_url(
#     KEYWORDS_STR, CATEGORY_CODE1, ORDER="relevance"
# )  # PAGE_NUM 省略，預設為 1
# print(url_1, "\n")

# # 2. 無關鍵字, 多個類別, 最新日期排序, 第 2 頁
# print("2. 無關鍵字, 多個類別, 最新日期排序, 第 2 頁:")
# url_2 = catch_1111_url("", CATEGORY_CODE2, ORDER="date", PAGE_NUM=2, USE_API=False)
# print(url_2, "\n")

# # 3. 有關鍵字, 無類別, 最新日期排序, 第 3 頁
# print("3. 有關鍵字, 無類別, 最新日期排序, 第 3 頁:")
# url_3 = catch_1111_url(KEYWORDS_STR, "", ORDER="date", PAGE_NUM=3, USE_API=False)
# print(url_3, "\n")

# # 4. 使用 API 生成網址
# print("4. 有關鍵字, 單一類別, 使用 API, 第 1 頁:")
# api_url_1 = catch_1111_url(
#     KEYWORDS_STR, CATEGORY_CODE1, ORDER="relevance", USE_API=True
# )
# print(api_url_1, "\n")

# # 5. 無關鍵字, 多個類別, 使用 API, 第 2 頁
# print("5. 無關鍵字, 多個類別, 使用 API, 第 2 頁:")
# api_url_2 = catch_1111_url("", CATEGORY_CODE2, ORDER="date", PAGE_NUM=2, USE_API=True)
# print(api_url_2, "\n")

# # 6. 有關鍵字, 無類別, 使用 API, 第 3 頁
# print("6. 有關鍵字, 無類別, 使用 API, 第 3 頁:")
# api_url_3 = catch_1111_url(KEYWORDS_STR, "", ORDER="date", PAGE_NUM=3, USE_API=True)
# print(api_url_3, "\n")

In [4]:
import requests
import pandas as pd
from tqdm.auto import tqdm
import urllib3
from typing import Optional, List, Union


def get_1111_api_data(
    KEYWORDS: str,
    CATEGORY: Union[str, List[str]],
    ORDER: str = "date",
    max_page: Optional[int] = None,
) -> pd.DataFrame:
    """
    從 1111 求職網高效擷取指定搜尋條件下的所有職缺資料。
    1.  將首次請求與總頁數獲取合併，減少 API 呼叫。
    2.  提供更穩健的錯誤處理機制，能跳過請求失敗的頁面。
    3.  程式碼結構簡潔，使用進度條顯示抓取進度。

    Args:
        KEYWORDS (str): 職缺的關鍵字，例如 "雲端工程師"。
        CATEGORY (Union[str, List[str]]): 職缺的類別代碼，例如 "140100" 或 ["140100", "140200"]。
        ORDER (str, optional): 排序方式。可選值為 "relevance" (相關性) 或 "date" (最新日期)。預設為 "date"。
        max_page (Optional[int], optional): 欲抓取的最大頁數。若為 None，則自動抓取所有頁面。預設為 None。

    Returns:
        pd.DataFrame: 包含所有職缺資料的 DataFrame。如果抓取失敗或無資料，則返回空的 DataFrame。
    """

    all_data = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    # 請求第一頁，同時獲取總頁數
    try:
        url = catch_1111_url(KEYWORDS, CATEGORY, ORDER, PAGE_NUM=1, USE_API=True)
        response = requests.get(url, headers=headers, verify=False, timeout=15)
        response.raise_for_status()
        data = response.json()

        hits = data.get("result", {}).get("hits", [])
        if not hits:
            print("找不到任何相關職缺資料。")
            return pd.DataFrame()

        all_data.extend(hits)
        totalPage = data.get("result", {}).get("pagination", {}).get("totalPage", 1)

    except (
        requests.exceptions.RequestException,
        requests.exceptions.JSONDecodeError,
    ) as e:
        print(f"抓取第一頁時發生錯誤，無法繼續: {e}")
        return pd.DataFrame()

    # 決定最終要抓取的頁數
    limit_page = totalPage
    if max_page is not None and max_page > 0:
        limit_page = min(totalPage, max_page)

    if limit_page <= 1:
        return pd.json_normalize(all_data)

    # 迭代抓取剩餘頁面 (從第 2 頁開始)
    for page_num in tqdm(range(2, limit_page + 1), desc="正在擷取資料"):
        try:
            url = catch_1111_url(
                KEYWORDS, CATEGORY, ORDER, PAGE_NUM=page_num, USE_API=True
            )
            response = requests.get(url, headers=headers, verify=False, timeout=15)
            response.raise_for_status()

            page_data = response.json().get("result", {}).get("hits", [])
            if not page_data:
                print(f"\n在第 {page_num} 頁後已無更多資料，提前結束。")
                break

            all_data.extend(page_data)

        except (
            requests.exceptions.RequestException,
            requests.exceptions.JSONDecodeError,
        ) as e:
            print(f"\n抓取第 {page_num} 頁時發生錯誤，已跳過: {e}")
            continue

    if not all_data:
        return pd.DataFrame()

    return pd.json_normalize(all_data)


# --- 使用範例 ---
KEYWORDS = "雲端工程師"
CATEGORY = ["140100", "140200"]

df_data = get_1111_api_data(KEYWORDS, CATEGORY, max_page=2)
# df_data = get_1111_data(KEYWORDS, CATEGORY)
df_data.head(1)

正在擷取資料:   0%|          | 0/1 [00:00<?, ?it/s]

,updateAt,jobId,companyId,companyName,description,title,role,remind,replyInDays,salary,...,require.certificates,require.experience,require.grades,require.majors,industry.id,industry.name,workCity.id,workCity.name,highlight.title,highlight.description
0,2025/06/17 11:24:00,113019436,48931667,富邦媒體科技股份有限公司(富邦momo),1.主要開發與維護 會員標籤/行銷/推薦應用相關平台\n2.開發數據資料處理平台，解決大量資...,D4000 JAVA資料工程師 【台中】,[1],[0],0,面議（經常性薪資達4萬元或以上）,...,[],3,"[16, 32, 64]","[130100, 130600, 0]",250214,百貨相關,100906,台中市北屯區,D4000 JAVA資料<em>工程師</em> 【台中】,1.主要開發與維護 會員標籤/行銷/推薦應用相關平台\n2.開發數據資料處理平台，解決大量資...


In [5]:
# 從指定的職缺網址獲取職缺的相關數據

def get_1111_page_data(job_url):
    """
    解析 1111 新版職缺頁面的 HTML 內容，並回傳結構化資料。

    參數:
    job_url (str): 職缺頁面的 URL。

    回傳:
    dict: 包含職缺資訊的結構化資料，包括職務名稱、公司名稱、工作性質、薪資待遇、學歷要求、上班地點、福利資訊及聯絡資訊等。
    """

    response = requests.get(job_url, verify=False)
    soup = BeautifulSoup(response.content, "html.parser")

    data = {}

    data["job_url"] = job_url

    # --- 1. 頁首區塊 (Top Section) ---
    header_section = soup.select_one(
        "section[data-v-e57f1019] > div.container > div.text-gray-600"
    )
    if header_section:
        data["職務名稱"] = header_section.select_one("h1").get_text(strip=True) or "N/A"
        data["公司名稱"] = (
            header_section.select_one("h2.inline").get_text(strip=True) or "N/A"
        )

        pills = header_section.select("div.flex.flex-wrap.mt-4.gap-3 > div")
        top_info = [p.get_text(strip=True, separator=" ") for p in pills]
        if len(top_info) >= 4:
            data["工作性質"], data["薪資待遇"], data["學歷要求"], data["上班地點"] = (
                top_info[:4]
            )

        info_items = header_section.select("ul.info-item > li")
        for item in info_items:
            key_tag = item.select_one("h3")
            val_tag = item.select_one("span") or item.select_one("time")
            if key_tag and val_tag:
                data[key_tag.get_text(strip=True)] = val_tag.get_text(strip=True)

    # --- 2. 主要內容區塊 (Main Content Sections) ---
    sections = soup.select("section[id]")
    for section in sections:
        section_title_tag = section.select_one("h2.text-lg.text-main")
        if not section_title_tag:
            continue

        section_title = section_title_tag.get_text(strip=True)
        section_data = {}

        # --- 標準內容區塊的通用解析 ---
        for div in section.select("div.content"):
            key_tag = div.find("h3")
            if key_tag:
                key = key_tag.get_text(strip=True)
                value_container = key_tag.find_next_sibling()
                if value_container:
                    list_items = value_container.select("li")
                    section_data[key] = (
                        [
                            li.get_text(strip=True, separator=" ")
                            .replace("、", "")
                            .strip()
                            for li in list_items
                        ]
                        if list_items
                        else value_container.get_text(separator="\n", strip=True)
                    )

        # --- 福利資訊的特殊結構處理 ---
        if section_title == "福利資訊":
            legal_welfare_h3 = section.find(
                "h3", string=lambda t: t and "法定福利" in t
            )
            if legal_welfare_h3:
                container = legal_welfare_h3.find_next_sibling("div")
                section_data["法定福利"] = [
                    p.get_text(strip=True) for p in container.select("p")
                ]

            company_welfare_h3 = section.find(
                "h3", string=lambda t: t and "公司福利" in t
            )
            if company_welfare_h3:
                welfare_map = {}
                categories = (
                    company_welfare_h3.find_parent("div")
                    .find_next_sibling("div")
                    .find_all("div", recursive=False, class_="flex")
                )
                for cat_div in categories:
                    cat_title = (
                        cat_div.select_one("h4").get_text(strip=True)
                        if cat_div.select_one("h4")
                        else "未分類"
                    )
                    items = [
                        p.get_text(strip=True)
                        for p in cat_div.select("div.flex-wrap p")
                    ]
                    welfare_map[cat_title] = items
                section_data["公司福利"] = welfare_map

            more_info_h3 = section.find("h3", string=lambda t: t and "更多說明" in t)
            if more_info_h3:
                container = more_info_h3.find_next_sibling("div")
                section_data["更多說明"] = (
                    container.get_text(separator="", strip=True)
                    .split("\n展開全部")[0]
                    .strip()
                )

        if section_data:
            data[section_title] = section_data

    # --- 單獨處理聯絡資訊 ---
    contact_section = soup.select_one("section#CONTACT_INFO")
    if contact_section:
        contact_data = {}
        contact_person_tag = contact_section.find(
            "h3", string=lambda t: t and "聯絡人員" in t
        )
        if contact_person_tag:
            contact_data["聯絡人員"] = contact_person_tag.find_next_sibling(
                "p"
            ).get_text(strip=True)
        if contact_data:
            data["聯絡資訊"] = contact_data
    # df = pd.json_normalize(data)
    return data


# 測試範例
job_url = "https://www.1111.com.tw/job/113019436"
get_1111_page_data(job_url)


{'job_url': 'https://www.1111.com.tw/job/113019436',
 '職務名稱': 'D4000 JAVA資料工程師 【台中】',
 '公司名稱': '富邦媒體科技股份有限公司(富邦momo)',
 '需求人數': '1  ~ 2 人',
 '到職日期': '一個月內',
 '更新日期': '2025 / 06 / 17',
 '工作內容': {'職缺描述': '1.主要開發與維護 會員標籤/行銷/推薦應用相關平台\n2.開發數據資料處理平台，解決大量資料處理時所衍生之效能與維護問題\n3.雲端平台開發及維運(GCP/AWS)\n4.Data Pipeline規劃、設計、開發與監控維護\n5.研發大數據架構相關技術，優化現有架構並導入新技術\n收合內容',
  '工作待遇': '面議 (經常性薪資達4萬元或以上)\n查看薪資水平',
  '職務類別': ['演算法開發工程師', '軟體工程師', '網站程式設計師'],
  '工作性質': ['全職'],
  '工作時間': ['日班'],
  '工作地點': '台中市\n                    北屯區\n                    文心路四段955號9樓'},
 '要求條件': {'學歷要求': ['大學以上'],
  '科系要求': ['數學統計學門', '電算機學門'],
  '工作經驗': '1 年以上經驗',
  '外語能力': ['英文 聽｜ 中等 説｜ 中等 讀｜ 中等 寫｜ 中等'],
  '工作技能': '不拘',
  '附加條件': '1.一年以上Java的軟體開發經驗\n2.具有 NoSQL (Redis、elasticsearch)的實務經驗\n3.具有SQL語法且具備Oracle、Postgre實務經驗\n4.具有Message Queue (例 Kafka、RabbitMQ)的實務經驗\n5.具有Git、Jenkis、Airflow 等Data Pipeline 開發實務經驗\n6.熟悉Linux作業系統操作\n\n* 如經面試錄取後，報到時須繳交「體格檢查表(一般)」，請於報到前自行至醫療機構辦理體檢 *\n展開全部',
  '歡迎身份': ['原住民']},
 '公司資訊': {'公司名稱': '富邦媒體科技股份有限公

In [6]:
# 根據關鍵字與職業類別 獲取所有工作職位的資料

SEARCH_TIMESTAMP = time.strftime("%Y-%m-%d", time.localtime(time.time()))
JOBCAT_CODE = "140100"
KEYWORDS = "雲端工程師"
FILE_NAME = f"({SEARCH_TIMESTAMP})_{WEB_NAME}_{KEYWORDS}_{JOBCAT_CODE}"
MAX_WORKERS = 5  # 同時運行的執行緒數量

print(f"開始執行 {FILE_NAME}")
df_api_data = get_1111_api_data(KEYWORDS, JOBCAT_CODE)
df_api_data["company_url"] = "https://www.1111.com.tw/corp/" + df_api_data["companyId"].astype(str)
df_api_data["job_url"] = "https://www.1111.com.tw/job/" + df_api_data["jobId"].astype(str)


# 使用 ThreadPoolExecutor 進行並行處理
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:  # 可以根據需要調整 max_workers 的數量
    list_page_data = list(tqdm(executor.map(get_1111_page_data, df_api_data['job_url']), total=len(df_api_data['job_url']), desc="Parsing Job URLs"))

df_page_data = pd.json_normalize(list_page_data)
all_jobs_df = pd.merge(df_api_data, df_page_data, on='job_url', how='left')

print(all_jobs_df.shape)

all_jobs_df.head(1)

開始執行 (2025-06-17)_1111_人力銀行_雲端工程師_140100


正在擷取資料:   0%|          | 0/14 [00:00<?, ?it/s]

Parsing Job URLs:   0%|          | 0/437 [00:00<?, ?it/s]

(437, 77)


,updateAt,jobId,companyId,companyName,description,title,role,remind,replyInDays,salary,...,工作內容.實習時段,要求條件.打字速度,工作內容.發展遠景,工作內容.成就樂趣,公司資訊.品牌名稱,工作內容.遠距工作,工作性質,薪資待遇,學歷要求,上班地點
0,2025/06/17 12:46:00,99025652,2169,經濟實業股份有限公司,1.泵浦發電機大數據輔助設計與運維之智慧雲端平台開發相關工作\n2.泵浦、發電機運轉中數據收...,軟體分析工程師(總公司),[1],[0],0,"月薪 45,000元~55,000元",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# all_jobs_df.to_csv (f"{FILE_NAME}.csv", index=False, encoding='utf-8-sig')
# print (f"已將所有職缺資料儲存到 {FILE_NAME}.csv")

all_jobs_df.to_excel(f"{FILE_NAME}.xlsx", index=False)
print(f"已將所有職缺資料儲存到 {FILE_NAME}.xlsx")

已將所有職缺資料儲存到 (2025-06-17)_1111_人力銀行_雲端工程師_140100.xlsx


In [8]:
all_jobs_df.columns

Index(['updateAt', 'jobId', 'companyId', 'companyName', 'description', 'title',
       'role', 'remind', 'replyInDays', 'salary', 'recruitCount', 'mrtId',
       'mrtTime', 'mrtNear', 'benefits', 'companyTags', 'isHappiness',
       'internship', 'jobType', 'bookmarked', 'hasMedias', 'isTop',
       'hasCompanyLogo', 'require.drivingLicense', 'require.certificates',
       'require.experience', 'require.grades', 'require.majors', 'industry.id',
       'industry.name', 'workCity.id', 'workCity.name', 'highlight.title',
       'highlight.description', 'company_url', 'job_url', '職務名稱', '公司名稱',
       '應徵人數', '需求人數', '到職日期', '更新日期', '工作內容.職缺描述', '工作內容.工作待遇', '工作內容.職務類別',
       '工作內容.工作性質', '工作內容.工作時間', '工作內容.休假制度', '工作內容.工作地點', '要求條件.學歷要求',
       '要求條件.科系要求', '要求條件.工作經驗', '要求條件.外語能力', '要求條件.方言能力', '要求條件.工作技能',
       '要求條件.歡迎身份', '公司資訊.公司名稱', '公司資訊.產業類別', '公司資訊.公司人數', '公司資訊.資本額',
       '聯絡資訊.聯絡人員', '要求條件.附加條件', '要求條件.電腦專長', '要求條件.具備駕照', '要求條件.自備車輛',
       '要求條件.專業證照', '要求條件.其他說明', '工作內